# OpenStreetMap (OSM) Data Wrangling Project

## Map area 

For this project, I have chosen to clean and explore the OpenStreetMap data of Boston, where I currently reside:  
- https://www.openstreetmap.org/relation/2315704
- http://metro.teczno.com/#boston

In [20]:
import sqlite3
import pandas as pd 
import re
db = sqlite3.connect("OSMBoston.db")
c = db.cursor()

## Issues with map data

After downloading a sample of the OSM data, converting them to csv files using prep_data.py, and uploading them into a SQLite database, I noticed a number of issues with the map data that warrant our attention: 
- Overabbreviated street names
- Inconsistent state names
- Inconsistent city names 
- Inconsistent and problematic postal codes 
- Addresses input in different fields

Some of these issues are programtically fixed using the additional procedure 'clean_element' in prep_data_clean.py. The cleaned version of the tables are also uploaded to the database with a suffix '_cleaned' for comparison with the original. 

### Overabbreviated street names  

While most street names are properly named, ending in "Street", "Avenue", "Drive" etc., others are overabbreivated. For instance, we see many occurences of "St.", "Ave", and "Ave." 

In [22]:
QUERY = """
        SELECT value, COUNT(*) 
        FROM (SELECT * FROM way_tags UNION ALL SELECT * FROM node_tags) t
        WHERE type = 'addr' AND key = 'street' 
        GROUP BY value ORDER BY value;
        """
c.execute(QUERY)
rows = c.fetchall()
st_names = pd.DataFrame(rows, columns = ['st_name', 'count'])

st_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
st_types_good = ["Street", "Avenue", "Drive", "Square", "Broadway", "Place", 
                 "Park", "Center", "Road", "Way", "Boulevard", "Lane"]  
st_types_other = {}

for name in st_names['st_name']: 
    st_type = st_type_re.search(name).group() 
    if st_type not in st_types_good: 
        if st_type in st_types_other: 
            st_types_other[st_type] += 1 
        else: 
            st_types_other[st_type] = 1

st_types_other

{u'1100': 1,
 u'1702': 1,
 u'3': 1,
 u'303': 1,
 u'6': 1,
 u'846028': 1,
 u'Ave': 7,
 u'Ave.': 4,
 u'Boylston': 1,
 u'Cambrdige': 1,
 u'Federal': 1,
 u'Fenway': 1,
 u'Floor': 1,
 u'Hall': 1,
 u'Hampshire': 1,
 u'Highway': 1,
 u'Hwy': 1,
 u'LEVEL': 1,
 u'Lafayette': 1,
 u'Mall': 1,
 u'Newbury': 1,
 u'Pl': 1,
 u'Row': 1,
 u'ST': 1,
 u'South': 1,
 u'Sq.': 1,
 u'St': 19,
 u'St.': 13,
 u'Terrace': 1,
 u'Wharf': 2,
 u'Windsor': 1,
 u'Winsor': 1,
 u'ave': 1,
 u'floor': 2,
 u'st': 1,
 u'street': 1}

For sake of standardization, we want to spell out these abbreviated names in full. Using the mapping table below, we cleaned up some of these abbreviated names in the cleaned dataset. 

In [36]:
st_types_mapping = {"Ave": "Avenue", 
                   "Ave.": "Avenue", 
                   "ave": "Avenue",
                   "Hwy": "Highway", 
                   "Pl": "Place", 
                   "ST": "Street", 
                   "Sq.": "Square", 
                   "St": "Street", 
                   "St.": "Street", 
                   "st": "Street", 
                   "street": "Street",
                   } 

QUERY = """
        SELECT wt.id, wt.key, wt.value, wtc.value
        FROM way_tags wt JOIN way_tags_clean wtc
        ON wt.id = wtc.id AND wt.key = wtc.key
        WHERE wt.key = "street" AND wt.value != wtc.value
        """
c.execute(QUERY)
rows = c.fetchall()
st_names_cleaned = pd.DataFrame(rows, columns = ['id', 'key', 'value_original', 'value_cleaned'])
st_names_cleaned.head()

,id,key,value_original,value_cleaned
0,29532447,street,Somerville Ave,Somerville Avenue
1,29532468,street,Somerville Ave.,Somerville Avenue
2,29532471,street,Somerville Ave.,Somerville Avenue
3,29537969,street,Newton ST,Newton Street
4,29554005,street,Tremont St.,Tremont Street


### Inconsistent state names

The state name of Massachusetts was inconsistently listed in the data surveyed. While 'MA' was the most common spelling, other entries listed it as 'MA- MASSACHUSETTS', 'Massachusetts', 'ma', and 'Ma'. In the cleaned dataset, we cleaned up these alternative spellings and standardized them as 'MA'. 

In [41]:
QUERY = """
        SELECT value, COUNT(*) FROM 
        (SELECT * FROM way_tags UNION ALL SELECT * FROM node_tags) t
        WHERE type = 'addr' AND key = 'state'
        GROUP BY value ORDER BY COUNT(*) DESC
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['state', 'count'])

,state,count
0,MA,665
1,MA- MASSACHUSETTS,62
2,Massachusetts,11
3,ma,2
4,Ma,1


### Inconsistent city names 

Along the same vein, cities are also inconsistently named in the map data. For example, there are several instances where the state names were listed along with the city names, and other instances where the city names were in all-upper or all-lower cases rather than the proper case. 

In [42]:
QUERY = """
        SELECT value, COUNT(*) 
        FROM (SELECT * FROM way_tags UNION ALL SELECT * FROM node_tags) t
        WHERE type = 'addr' AND key = 'city'
        GROUP BY value ORDER BY COUNT(*) DESC
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['city', 'count'])

,city,count
0,Cambridge,347
1,Boston,260
2,Somerville,32
3,Brookline,9
4,"Boston, MA",6
5,Charlestown,6
6,"Cambridge, MA",5
7,"Cambridge, Massachusetts",5
8,Roxbury Crossing,2
9,South Boston,2


We want to standardize all city names to proper case, without accompanying state names. The 'clean_element' procedure does this by ignoring portions of the string that comes after any comma, as well as applying str.title() to the values. The result is a much cleaner list of cities. 

In [38]:
QUERY = """
        SELECT value, COUNT(*) 
        FROM (SELECT * FROM way_tags_clean UNION ALL SELECT * FROM node_tags_clean) t
        WHERE type = 'addr' AND key = 'city'
        GROUP BY value ORDER BY COUNT(*) DESC
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['city_cleaned', 'count'])

,city_cleaned,count
0,Cambridge,357
1,Boston,268
2,Somerville,33
3,Brookline,9
4,Charlestown,6
5,Roxbury Crossing,2
6,South Boston,2
7,East Boston,1
8,Roxbury,1
9,South End,1


### Problematic postal codes

Not surprisingly, we also see inconsistency in postal codes. While most postal codes use the 5-digit convention, others use the full 9-digit convention. Worse, there are instances where state was included in the postal code. 

In [39]:
QUERY = """
        SELECT tags.value, COUNT(*) AS count 
        FROM (SELECT * FROM node_tags UNION ALL SELECT * FROM way_tags) tags
        WHERE tags.key = 'postcode' 
        GROUP BY tags.value ORDER BY count DESC LIMIT 30; 
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['zipcode', 'count'])

,zipcode,count
0,02139,282
1,02114,61
2,02215,50
3,02116,42
4,02138,34
5,02142,32
6,02143,30
7,02210,24
8,02111,17
9,02141,16


For sake of standardization, we will remove non-numerical characters and keep only the first 5 digits as zipcodes. To do so, we used regex to replace all non-numeric characters with empty string, and used string-slicing to retain only the first 5 digits as zipcode. Below is a sample of zipcodes that were altered: 

In [51]:
QUERY = """
        SELECT wt.id, wt.key, wt.value, wtc.value
        FROM way_tags wt JOIN way_tags_clean wtc
        ON wt.id = wtc.id AND wt.key = wtc.key
        WHERE wt.key = "postcode" AND wt.value != wtc.value
        LIMIT 8 OFFSET 15
        """
c.execute(QUERY)
rows = c.fetchall()
st_names_cleaned = pd.DataFrame(rows, columns = ['id', 'key', 'value_original', 'value_cleaned'])
st_names_cleaned

,id,key,value_original,value_cleaned
0,29935638,postcode,02138-2901,02138
1,29937220,postcode,02138-2701,02138
2,151417813,postcode,MA,
3,167022743,postcode,02138-2801,02138
4,212292804,postcode,02138-2706,02138
5,288480553,postcode,MA 02116,02116
6,305490725,postcode,02138-2933,02138
7,305490726,postcode,02138-2933,02138


### Addresses input in different fields

The best practice of inputing addresses in OSM is to input various components of the address (e.g. city, housenumber, postcode, street, etc.) as individual key-value pairs, as demonstrated in the example below: 

In [70]:
QUERY = """
        SELECT *
        FROM (SELECT * FROM node_tags UNION ALL SELECT * FROM way_tags) tags
        WHERE tags.id = '69504742' AND type = 'addr'
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns = ['id', 'key', 'value', 'type'])

,id,key,value,type
0,69504742,city,Boston,addr
1,69504742,housenumber,145,addr
2,69504742,postcode,02116,addr
3,69504742,state,MA,addr
4,69504742,street,Dartmouth Street,addr


While most entries appear to adhere to this practice, there are also a significant number that violated this practice. Instead, the entire address was input as a single key-value pair under the key 'address'. Below are some of these examples. The 'clean_element' procedure does not address this issue, but to do so, we would need to create 5 new key-value pairs for tag listed with the 'address' key, effectively splitting up the 'address' value into its various components. 

In [61]:
QUERY = """
        SELECT * FROM (SELECT * FROM node_tags UNION ALL SELECT * FROM way_tags) tags
        WHERE key = 'address' LIMIT 5
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns = ['id', 'key', 'value', 'type'])

,id,key,value,type
0,257486704,address,"200 Nashua Street, Boston, MA 02114",regular
1,257486710,address,"20 Bradston Street, Boston, MA 02118",regular
2,257486735,address,"40 Thorndike Street, Cambridge, MA 02141",regular
3,257489554,address,"125 Parker Hill Ave., Boston, MA",regular
4,257489564,address,"243 Charles Street, Boston, MA",regular


## Data Overview and Additional Ideas

This section contains basic statistics about the dataset, the SQL queries used to gather them, as well as some additional ideas about improving the dataset and deploying it for other purposes. 

### File Size 

In [123]:
file_size = {'boston.osm': '53.5MB', 'nodes_tags': '2.6MB', 'nodes.csv': '17.5MB',
            'ways_nodes.csv': '6.3MB', 'ways_tags.csv': '3.6MB', 'ways.csv': '2.0MB',
             'OSMBoston.db': '34.9MB', }
file_size_df = pd.DataFrame.from_dict(file_size, orient='index')
file_size_df.rename(columns={0: 'file_size'}, inplace = True)
file_size_df

,file_size
boston.osm,53.5MB
ways_nodes.csv,6.3MB
nodes.csv,17.5MB
OSMBoston.db,34.9MB
ways.csv,2.0MB
nodes_tags,2.6MB
ways_tags.csv,3.6MB


### Number of Nodes

In [69]:
QUERY = "SELECT COUNT(*) FROM nodes"
c.execute(QUERY)
print c.fetchone()[0]

216691


### Number of Ways 

In [17]:
QUERY = "SELECT COUNT(DISTINCT id) FROM ways"
c.execute(QUERY)
print c.fetchone()[0]

31551


### Number of Unique Users 

A total of 636 unique users had contributed to this dataset, which is pretty impressive. 

In [32]:
QUERY = """
        SELECT COUNT(DISTINCT uid) 
        FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) t  
        """
c.execute(QUERY)
print c.fetchone()[0]

636


### Top Contributors

However, more than three quarters of the contributions came from just three users, with the most prolific user 'crschmidt' accounting for over 50 percent of all the contributions. In general, open source platforms could benefit from significant participation from more parties, rather than rely on the contributions of a few individuals. As such, OSM may want to consider implementing some gamification elements or other incentives to encourage more users to actively participate in making OSM a more complete and robust mapping database.

In [54]:
QUERY = """
        SELECT user, 
               COUNT(*), 
               ROUND(CAST(COUNT(*) AS FLOAT) * 100 / (SELECT COUNT(*) FROM nodes UNION ALL SELECT user FROM ways), 1)   
        FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) t
        GROUP BY user ORDER BY COUNT(*) DESC LIMIT 10
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['user', 'contrib_count', 'contrib_percent'])

,user,contrib_count,contrib_percent
0,crschmidt,117658,54.3
1,wambag,25220,11.6
2,jremillard-massgis,24180,11.2
3,mapper999,14139,6.5
4,morganwahl,12139,5.6
5,OceanVortex,9253,4.3
6,MassGIS Import,3962,1.8
7,JasonWoof,3663,1.7
8,Ahlzen,2396,1.1
9,fiveisalive,2145,1.0


### Amenity Types 

The table below shows the top amenity categories tagged in OpenStreetMaps. Not surprisingly, 'restaurant' was the most common category. However, we also see related categories like 'cafe', 'fast_food', 'pub', and 'bar' listed separately. One might argue that there is little distinction across these categories (especially between 'pub' and 'bar') so they should all be listed under a broader 'restaurant & bar' category - if not as a single category then an umbrella one (similar to 'addr' as in 'addr:street'). More on this in a later section.

In [77]:
QUERY = """
        SELECT value, COUNT(*) FROM node_tags WHERE key = 'amenity' 
        GROUP BY value ORDER BY COUNT(*) DESC LIMIT 15
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['value', 'count'], index=range(1,16))

,value,count
1,restaurant,291
2,bench,228
3,bicycle_parking,152
4,library,140
5,school,130
6,cafe,124
7,bicycle_rental,99
8,place_of_worship,88
9,fast_food,87
10,fountain,57


### Cuisine Types 

Having said that, let's explore the cuisine types of all the nodes tagged as 'restaurant'. Usual suspects like 'pizza' and 'American' rank amongst the top cuisine types, but it was somewhat surprising to see 'Mexican' as the top category as Boston doesn't have a particularly strong Mexican influence. Instead, I would have expected to see 'Italian' rank higher given the city's heritage. 

In [88]:
QUERY = """
        SELECT value, COUNT(*) FROM node_tags 
        WHERE id IN (SELECT id FROM node_tags WHERE key = 'amenity' and value = 'restaurant') 
              AND key = 'cuisine'
        GROUP BY value ORDER BY COUNT(*) DESC LIMIT 10
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns=['value', 'count'], index=range(1,11))

,value,count
1,mexican,16
2,pizza,15
3,american,13
4,italian,13
5,chinese,11
6,indian,11
7,thai,9
8,japanese,8
9,asian,7
10,international,7


### Schools

Last but not least, I am also interested to see how many schools identified in the data. True to its reputation as one of the most educated cities in the U.S. with a fine education system, Boston is home to many colleges and other institutions of learning. In our dataset, we see a total of 123 institutions listed, ranging from elementary schools to graduate schools. 

In [92]:
QUERY = """
        SELECT DISTINCT value FROM node_tags 
        WHERE id IN (SELECT id FROM node_tags WHERE key = 'amenity' and value = 'school') AND key = 'name'
        ORDER BY value 
        """
c.execute(QUERY)
rows = c.fetchall()
pd.DataFrame(rows, columns = ['school'])

,school
0,Abiel Smith School
1,Advent
2,Another Course To College
3,Another Course to College School
4,Baxter School
5,Blackstone School
6,Blaine School
7,Boston Architectural College
8,Boston Arts Academy
9,Boston Center for Adult Education


## Potential improvements

As alluded to in the "amenity types" section earlier, OpenStreetMap could potentially do a better job at re-classifying related categories such as 'cafe', 'fast_food', 'pub', and 'bar' under a broader 'restaurant & bar' category, if not as a single category then as an umbrella one similar to 'addr' in 'addr:street'. This will be helpful for users mining all F&B establishments in an area. Instead of pain-stakingly combing through all categories to avoid missing a relevant category, users may simply search for tags within that one category, or top-level category. Making that change should not be technically difficult (just remapping of existing key values and restricting user options to pick amenity types going forward), though it may make the actual querying of data slightly more cumbersome, as we had experienced in this exercise, and more importantly, raise questions about to what degree should roll-up categories be deployed which in turn have larger implications on the OSM data structure. 

In addition to improving the data schema, OpenStreetMap could also consider requiring users to adhere more strictly to standardized best practices when tagging common elements like addresses (e.g. typing "Street" instead of "St.", using "MA" instead of "Ma" etc.). This will help improve the quality of the dataset and reduce the effort required for analysts to parse through and clean up the data afterward. However, such rules may be complex to implement given the multitude of taggable elements, as well as variations in commonly-accepted conventions across different local cultures. If not executed well, the more restrictive requirements may result in unpleasant user experience for the contributers, in turn curbing their enthusiasm at participating in the initiative. As such, OpenStreetMap may want to consider implementing such requirements on a piece-meal basis, experimenting first with regions they know and understand well. 

## Conclusion 

In conclusion, the OSM data can certainly be cleaned up further, and could perhaps benefit from data schema changes and stricter requirements on user input. Nevertheless, it has been a very interesting exercise where I learnt to parse XML and programatically audit and clean some of these data issues identified. 